<a href="https://colab.research.google.com/github/Real1CM/MLassignment/blob/main/%E4%BD%9C%E4%B8%9A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

只跑了10轮，卡带不动

In [5]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# 创建CartPole环境
env = gym.make("CartPole-v1")

# Q值网络
def build_q_network(state_size, action_size):
    model = tf.keras.Sequential([
        layers.Dense(24, activation='relu', input_shape=(state_size,)),
        layers.Dense(24, activation='relu'),
        layers.Dense(action_size, activation='linear')  # 输出Q值
    ])
    return model

# Q-learning超参数
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
gamma = 0.99  # 折扣因子
epsilon = 0.1  # 探索率
learning_rate = 0.001
batch_size = 64
memory = []  # 存储经验回放

# 初始化Q网络和优化器
model = build_q_network(state_size, action_size)
optimizer = tf.keras.optimizers.Adam(learning_rate)

# 选择动作的函数（根据epsilon-greedy策略）
def select_action(state):
    if np.random.rand() <= epsilon:
        return np.random.choice(action_size)  # 探索
    q_values = model(state[np.newaxis])  # 使用模型预测Q值
    return np.argmax(q_values)  # 利用

# 存储经验的函数
def store_experience(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))
    if len(memory) > 10000:  # 限制经验回放的大小
        memory.pop(0)

# 从经验回放中采样并训练模型
def train_q_network():
    if len(memory) < batch_size:
        return
    batch = np.random.choice(len(memory), batch_size, replace=False)
    for idx in batch:
        state, action, reward, next_state, done = memory[idx]
        state = np.array(state, dtype=np.float32)
        next_state = np.array(next_state, dtype=np.float32)

        with tf.GradientTape() as tape:
            q_values = model(state[np.newaxis])  # 当前状态下所有动作的Q值
            q_value = q_values[0][action]  # 当前动作的Q值

            # 计算目标Q值
            if done:
                target_q_value = reward
            else:
                next_q_values = model(next_state[np.newaxis])
                target_q_value = reward + gamma * np.max(next_q_values)

            # 计算损失
            loss = tf.square(target_q_value - q_value)

        # 计算梯度并更新网络
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

# 训练过程
num_episodes = 10
for episode in range(num_episodes):
    state = env.reset()
    state = np.array(state, dtype=np.float32)
    total_reward = 0

    for t in range(200):
        action = select_action(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.array(next_state, dtype=np.float32)

        store_experience(state, action, reward, next_state, done)
        train_q_network()

        state = next_state
        total_reward += reward

        if done:
            print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}")
            break

Episode 1/10, Total Reward: 83.0
Episode 2/10, Total Reward: 13.0
Episode 3/10, Total Reward: 9.0
Episode 4/10, Total Reward: 11.0
Episode 5/10, Total Reward: 22.0
Episode 6/10, Total Reward: 9.0
Episode 7/10, Total Reward: 8.0
Episode 8/10, Total Reward: 9.0
Episode 9/10, Total Reward: 14.0
Episode 10/10, Total Reward: 13.0


In [6]:
# 测试模型
total_rewards = 0
for _ in range(1):  # 运行1个测试回合
    state = env.reset()
    state = np.array(state, dtype=np.float32)
    for t in range(200):
        action = select_action(state)  # 使用训练好的模型选择动作
        next_state, reward, done, _ = env.step(action)
        state = np.array(next_state, dtype=np.float32)
        total_rewards += reward
        if done:
            break

average_reward = total_rewards / 10
print(f"Average reward over 10 test episodes: {average_reward}")

Average reward over 10 test episodes: 9.4
